In [1]:
import os
import sys

import json

import numpy as np
import scanpy as sc
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report

import scarches as sca
from scarches.dataset.trvae.data_handling import remove_sparsity
import matplotlib.pyplot as plt

import torch

from dotenv import load_dotenv

from lightning.pytorch.loggers import WandbLogger
import wandb

import session_info
import warnings
from pyprojroot.here import here

#plt.style.use(['science','nature','no-latex'])
dpi_fig_save = 300
sc.set_figure_params(dpi=100, dpi_save=dpi_fig_save, vector_friendly=True)

# Setting some parameters
warnings.filterwarnings("ignore")

from sklearn.model_selection import StratifiedKFold

overwriteData = True
overwriteFigures = True

# Set random seed
random_seed = 42

import scvi
scvi.settings.dl_num_workers = 8
scvi.settings.seed = random_seed

import warnings
warnings.filterwarnings('ignore')

torch.set_float32_matmul_precision('high')
torch.set_printoptions(precision=3, sci_mode=False, edgeitems=7)
torch.multiprocessing.set_sharing_strategy('file_system')

%load_ext autoreload
%autoreload 2

/home/dmaspero/miniconda3/envs/scarches/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
 captum (see https://github.com/pytorch/captum).


INFO: Seed set to 42


INFO:lightning.fabric.utilities.seed:Seed set to 42


In [2]:
assert load_dotenv()

In [3]:
workingDir = here('04_visualizing_final_embedding_space/SCGT00_CentralizedDataset/')

In [4]:
class CustomWandbLogger(WandbLogger):
    @property
    def save_dir(self):
        """Gets the save directory.

        Returns:
            The path to the save directory.

        """
        return self.experiment.dir

In [5]:
scvi.__version__

'1.1.2'

# Loading data


In [6]:
# Load the h5ad file
adata = sc.read_h5ad(here("R3/01_fromDatasets_to_CellTypes/SCGT00_MAIN_annotated_LowQFilt.h5ad"))#, 
#                     backed='r+', chunk_size=50000)


**Loading gene universe**

In [7]:
geneUniverse = pd.read_pickle(here('03_downstream_analysis/02_gene_universe_definition/results/04_selected_gene_list.pkl'))
geneUniverse

,hgnc_id,symbol,locus_group,HUGO_status
ensembl_gene_id,,,,
ENSG00000000003,HGNC:11858,TSPAN6,protein_coding,official
ENSG00000000457,HGNC:19285,SCYL3,protein_coding,official
ENSG00000000938,HGNC:3697,FGR,protein_coding,official
ENSG00000000971,HGNC:4883,CFH,protein_coding,official
ENSG00000001036,HGNC:4008,FUCA2,protein_coding,official
...,...,...,...,...
ENSG00000278817,NaN,ENSG00000278817,protein_coding,non-official
ENSG00000278828,HGNC:4775,H3C10,protein_coding,official
ENSG00000280670,HGNC:27003,CCDC163,protein_coding,official


In [8]:
adata = adata[:,geneUniverse.index].copy()
adata

AnnData object with n_obs × n_vars = 756120 × 8253
    obs: 'studyID', 'libraryID', 'sampleID', 'chemistry', 'disease', 'sex', 'binned_age', 'batches', 'Level1'
    var: 'hgnc_id', 'symbol', 'locus_group', 'HUGO_status'

In [9]:
adata.obs.value_counts('Level1')

Level1
T_CD4_NonNaive    159203
Mono              131392
T_CD4_Naive       123116
T_CD8_NonNaive     93256
ILC                57046
B                  55869
Platelets          52472
T_CD8_Naive        40745
UTC                23420
DC                 10440
pDC                 3125
Cycling             2741
Progenitors         1716
Plasma              1383
RBC                  196
Name: count, dtype: int64

In [10]:
adata.obs.value_counts('disease')

disease
RA     353283
PS     114090
PSA     97885
UC      87969
CD      75809
SLE     27084
Name: count, dtype: int64

In [11]:
adata.obs.value_counts('batches')

batches
Chron_Pool2Pool4    57071
UC_Pool5Pool7       53229
RA_Pool19Pool20     48964
RA_Pool15Pool16     46909
RA_Pool13Pool14     46615
RA_Pool2Pool4       46206
PS_Pool3Pool5       45743
PS_Pool6Pool7       43414
RA_Pool1Pool3       42631
RA_Pool10Pool12     42216
RA_Pool9Pool11      40449
RA_Pool6Pool8       39293
PsA_Pool5Pool7      35860
UC_Pool6Pool8       34740
PsA_Pool1Pool3      34428
PsA_Pool2Pool4      27597
SLE_Pool2Pool4      27084
PS_Pool2Pool4       24933
IBD_Pool1Pool2      18738
Name: count, dtype: int64

In [12]:
adata.obs['binned_age'] = adata.obs['binned_age'].astype(str)

In [13]:
genes = adata.var

In [14]:
# Retrieve MT and RB genes present in the dataset
MT_gene_idx = genes["symbol"].str.startswith("MT-")
print(f"{np.sum(MT_gene_idx)} mitochondrial genes")

RB_gene_idx = genes["symbol"].str.startswith(("RPS", "RPL"))
print(f"{np.sum(RB_gene_idx)} ribosomal genes")

# Retrieve TCR and BCR present in the dataset
TCR_gene_idx = genes["symbol"].str.contains("^TRA(J|V)|^TRB(J|V|D)")
print(f"{np.sum(TCR_gene_idx)} TCR genes")

BCR_gene_idx = genes["symbol"].str.contains("^IGH(J|V)")
print(f"{np.sum(BCR_gene_idx)} TCR genes")

HB_gene_idx = genes["symbol"].str.contains("^HB[^(P)]")
print(f"{np.sum(HB_gene_idx)} HB genes")

# Some of those genes will be included anyway because are part of curated gene sets.
MHC_gene_idx = genes["symbol"].str.contains("^HLA-")
print(f"{np.sum(MHC_gene_idx)} MHC genes")

PLT_gene_idx = genes["symbol"].isin(["PPBP", "PDGF", "ANG1", "LAPTM4B", "WASF3", "TPM3", "PF4", "TAC1"])
print(f"{np.sum(PLT_gene_idx)} PLT genes")

0 mitochondrial genes
0 ribosomal genes
0 TCR genes
0 TCR genes
1 HB genes
20 MHC genes
3 PLT genes


**Parameters**

In [15]:
setup_kwargs = dict(
    layer=None, 
    batch_key='batches',
    categorical_covariate_keys = None,
    labels_key = 'Level1' # needed for the following scANVI fine tuning
)

scvi_kwargs = dict(n_hidden=256,
                   n_latent=30,
                   n_layers=4, 
                   gene_likelihood='nb')

trainer_kwargs = dict(
    checkpointing_monitor = 'elbo_validation',
    early_stopping_monitor = 'reconstruction_loss_validation',
    early_stopping_patience = 2,
    early_stopping_min_delta=0.1,
    early_stopping = True,
    max_epochs = 1000,

    #logger = # wandb
)
# https://docs.scvi-tools.org/en/stable/api/reference/scvi.train.TrainingPlan.html#scvi.train.TrainingPlan
plan_kwargs = dict(
    lr = 5e-4,
    #reduce_lr_on_plateau = True
)

# https://docs.scvi-tools.org/en/stable/api/reference/scvi.module.VAE.html#scvi.module.VAE
#vae = dict(
#    use_layer_norm='both',
#    use_batch_norm='none',
#    encode_covariates=True,
#    deeply_inject_covariates=False
#)
datasplitter_kwargs = dict(pin_memory=True)
scvi_parameter_dict = setup_kwargs | scvi_kwargs | trainer_kwargs | plan_kwargs| datasplitter_kwargs

In [16]:
run_name = f"MAINobj_scVI_pretraining_noRBCnPlat"
run_name

'MAINobj_scVI_pretraining_noRBCnPlat'

In [17]:
sca.models.SCVI.setup_anndata(adata, 
                              **setup_kwargs)

Outdated cuSPARSE installation found.
Version JAX was built against: 12200
Minimum supported: 12100
Installed version: 12002
The local installation version must be no lower than 12100. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [18]:
logger = CustomWandbLogger(name = run_name, project='inflammation_atlas_R1_scANVI_batches', config = scvi_parameter_dict)

In [19]:
scvi_model = sca.models.SCVI(adata, **scvi_kwargs)

In [20]:
scvi_model.train(logger=logger, datasplitter_kwargs=datasplitter_kwargs, plan_kwargs = plan_kwargs, **trainer_kwargs)

INFO: GPU available: True (cuda), used: True


INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True


INFO: TPU available: False, using: 0 TPU cores


INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores


INFO: IPU available: False, using: 0 IPUs


...

Epoch 26/1000:   2%|█                                       | 25/1000 [41:50<27:16:50, 100.73s/it, v_num=4tyk, train_loss_step=2.11e+3, train_loss_epoch=1.98e+3]

Epoch 26/1000:   3%|█                                       | 26/1000 [43:30<27:13:05, 100.60s/it, v_num=4tyk, train_loss_step=2.11e+3, train_loss_epoch=1.98e+3]

Epoch 26/1000:   3%|█                                       | 26/1000 [43:30<27:13:05, 100.60s/it, v_num=4tyk, train_loss_step=2.06e+3, train_loss_epoch=1.98e+3]

Epoch 26/1000:   3%|█                                       | 26/1000 [43:30<27:09:57, 100.41s/it, v_num=4tyk, train_loss_step=2.06e+3, train_loss_epoch=1.98e+3]


Monitored metric reconstruction_loss_validation did not improve in the last 2 records. Best score: 1973.550. Signaling Trainer to stop.


In [21]:
wandb.finish()

wandb:                                                                                


wandb: 
wandb: Run history:
wandb:                     elbo_train █▆▄▄▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
wandb:                elbo_validation █▅▄▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
wandb:                          epoch ▁▁▁▂▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
wandb:                kl_global_train ▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:           kl_global_validation ▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:                 kl_local_train ▇█▆▅▄▄▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
wandb:            kl_local_validation █▅▄▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:                      kl_weight ▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
wandb:      reconstruction_loss_train █▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb: reconstruction_loss_validation █▅▄▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
wandb:               train_loss_epoch █▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:                train_loss_step ▄▆▅▄█▆▆▃▅▇▁▆▃▄▄▇▃▅▅▄▃▃▃▃▂▅▂▃▁█▅▄▂▄▃▆▂▇▄▇
wandb:            trainer/global_step ▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
wandb:                validation_loss █▅▄▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:

wandb: 🚀 View run MAINobj_scVI_pretraining_noRBCnPlat at: https://wandb.ai/inflammation/inflammation_atlas_R1_scANVI_batches/runs/vi1j4tyk/workspace
wandb: Synced 6 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)


wandb: Find logs at: ./wandb/run-20241022_175043-vi1j4tyk/logs


In [22]:
if overwriteData:
    scvi_model.save(here(f"{workingDir}/results/scVI_model_pretreined_batches/"), 
               overwrite = True, 
               save_anndata = True)

In [23]:
scvi_emb = scvi_model.get_latent_representation(adata=adata)

In [24]:
np.savez_compressed(file = str(here(f"{workingDir}/results/scVI_model_pretreined_batches/scVI_embedding.npz")), arr=scvi_emb)